In [41]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pickle

# Laboratorio 8: Random Forest y despliegues

**Duración:** 2 horas  
**Formato:** Implementación, despliegue y competencia  

---

## Portada del equipo

**Integrantes:**
- Nombre 1 (Usuario GitHub)
- Nombre 2 (Usuario GitHub)
- Nombre 3 (Usuario GitHub)

**Repositorio del equipo:**  
<https://github.com/usuario/equipoX>

**Fecha de entrega:**  
__/__/____

## Elemento 1 - Implementación del Random Forest

In [42]:
df=pd.read_csv('iris_train.csv')
X,y=df.iloc[:,:-1].values,df.iloc[:,-1].values

In [43]:
class RandomForest:
  def __init__(self, n_estimators=100, max_depth=None, max_features='sqrt', random_state=17):
    self.n_estimators = n_estimators
    self.max_depth = max_depth
    self.max_features = max_features
    self.random_state = random_state
    self.trees = []

  def bootstrap(self, X, y):
    n_samples = len(X)
    idxs = np.random.choice(n_samples, n_samples, replace=True)
    return X[idxs], y[idxs]

  def fit(self, X, y):
    self.trees = []
    n_features = X.shape[1]

    if self.max_features == 'sqrt':
      self.max_features_num = max(1, int(np.sqrt(n_features)))
    elif self.max_features == 'log2':
      self.max_features_num = max(1, int(np.log2(n_features)))
    else:
      self.max_features_num = n_features

    # Handle max_depth based on string inputs 'sqrt' or 'log2'
    if self.max_depth == 'sqrt':
        self.max_depth_num = max(1, int(np.sqrt(n_features)))
    elif self.max_depth == 'log2':
        self.max_depth_num = max(1, int(np.log2(n_features)))
    else:
        self.max_depth_num = self.max_depth # Use the provided integer or None


    for i in range(self.n_estimators):
      tree = DecisionTreeClassifier(max_depth=self.max_depth_num, max_features=self.max_features_num, random_state=self.random_state + i)

      X_sample, y_sample = self.bootstrap(X, y)
      tree.fit(X_sample, y_sample)
      self.trees.append(tree)


  def predict(self, X):
    tree_preds = np.array([tree.predict(X) for tree in self.trees])
    # Use axis=0 for column-wise operation
    return np.array([np.argmax(np.bincount(tree_preds[:, i])) for i in range(tree_preds.shape[1])])


  def fit_predict(self, X, y):
    self.fit(X, y)
    return self.predict(X)

  def get_params(self, deep=True):
    return {'n_estimators': self.n_estimators, 'max_depth': self.max_depth, 'max_features': self.max_features, 'random_state': self.random_state}

  def set_params(self, **params):
    for key, value in params.items():
      setattr(self, key, value)
    return self

In [44]:
df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,123.000000,123.000000,124.000000,120.000000,125.000000
mean,5.821221,2.764442,3.909994,1.186667,0.984000
std,2.428445,2.174626,2.484749,0.758474,0.822898
min,-11.601111,-14.870849,1.100000,0.100000,0.000000
25%,5.100000,2.700000,1.600000,0.300000,0.000000
50%,5.700000,3.000000,4.250000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,24.111271,4.400000,23.439238,2.500000,2.000000


In [45]:
df[(df["sepal length (cm)"]>10) | (df["sepal length (cm)"]<0)]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
22,-11.601111,3.8,6.4,2.0,2
97,24.111271,2.3,4.4,1.3,1


In [46]:
# Importar knn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import KNNImputer

# Rellenar datos atípicos usando el promedio de su clase
df=pd.read_csv('iris_train.csv')
df[(df > 10) | (df < 0)] = None

# Rellenar nulos con KNN por clase
df_filled = df.copy()

for clase in df.iloc[:, -1].unique():
    mascara = df.iloc[:, -1] == clase
    imputer = KNNImputer(n_neighbors=5)
    df_filled.loc[mascara, df.columns[:-1]] = imputer.fit_transform(df.loc[mascara, df.columns[:-1]])


In [47]:
df_filled.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,125.000000,125.000000,125.000000,125.000000,125.000000
mean,5.820960,3.041760,3.753120,1.196160,0.984000
std,0.817912,0.446658,1.766489,0.754991,0.822898
min,4.300000,2.000000,1.100000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.700000,3.000000,4.200000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.700000,4.400000,6.900000,2.500000,2.000000


In [48]:
X,y=df_filled.iloc[:,:-1].values,df_filled.iloc[:,-1].values

rf=RandomForest(n_estimators=100, max_depth='sqrt', random_state=17)
rf.fit(X,y)

y_hat=rf.predict(X)
accuracy_score(y,y_hat)

0.96

In [49]:
with open("../models/modelo.pkl", "wb") as f:
    pickle.dump(rf, f)

### Elemento 1 - Preguntas teóricas

**¿Por qué el bagging ayuda a reducir la varianza del modelo?**

El bagging promedia las predicciones de los numerosos árboles de decisión, cada uno entrenado en una versión ligeramente diferente de los datos obtenidos con Bootstrap. Los árboles de decisión individuales son propensos a la alta varianza, lo que significa que pequeños cambios en los datos de entrenamiento pueden alterar drásticamente su estructura y predicciones, llevando al sobreajuste. Al entrenar cada árbol en una muestra Bootstrap distinta, se genera diversidad entre ellos, haciendo que cometan errores diferentes. Cuando se agregan estas predicciones, los errores específicos de cada árbol tienden a cancelarse, dando como resultado un modelo de ensamble final mucho más estable, menos sensible al ruido de la muestra de entrenamiento particular, y con una varianza general más baja, lo que mejora su capacidad de generalización a datos no vistos.

**¿Qué efecto tiene limitar el número de variables consideradas en cada división?**

Limitar el número de variables consideradas aumenta su diversidad. Al no permitir que variables muy predictivas dominen todas las divisiones en todos los árboles, se fuerza a cada árbol a explorar diferentes conjuntos de características y, por lo tanto, a aprender estructuras y reglas distintas. Esta menor correlación entre los árboles hace que el proceso de agregación (voto o promedio) sea más efectivo para cancelar los errores individuales, lo que resulta en una reducción significativa de la varianza del modelo final y una mejor capacidad de generalización. Adicionalmente, evaluar menos variables en cada nodo acelera el proceso de entrenamiento de cada árbol.

**¿Cómo cambia el desempeño al incrementar el número de árboles en el ensamble?**

Al incrementar el número de árboles en un Random Forest, el desempeño del modelo puede mejorar debido a una reducción en la varianza, ya que promediar o votar las predicciones de más árboles ayuda a cancelar errores. Sin embargo, esto no mejora infinitamente, ya que tiende a estabilizarse después de cierto número de árboles. Por lo tanto, añadir más árboles aumenta la complejidad sin agregar muchos beneficios adicionales en las métricas.

## Elemento 2 - Comparativa con scikit-learn

In [50]:
# comparar con sklearn
from sklearn.ensemble import RandomForestClassifier
rf_sklearn=RandomForestClassifier(n_estimators=100, max_depth=None, random_state=17)
rf_sklearn.fit(X,y)

y_hat=rf_sklearn.predict(X)
accuracy_score(y,y_hat)

1.0

In [51]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 150, 200,400],  # Number of trees in the forest
    'max_depth': [None, 5, 10, 15]  # Maximum depth of the trees
}

# Create a GridSearchCV object
# We use the custom RandomForest class
grid_search = GridSearchCV(RandomForest(random_state=17), param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the data
grid_search.fit(X, y)

# Print the best parameters and the best score
print("Best parameters: ", grid_search.best_params_)
print("Best accuracy: ", grid_search.best_score_)

Best parameters:  {'max_depth': None, 'n_estimators': 50}
Best accuracy:  0.944


In [52]:
from sklearn.metrics import confusion_matrix, classification_report

# Make predictions using the scikit-learn RandomForestClassifier
y_pred_sklearn = rf_sklearn.predict(X)

# Calculate and print the confusion matrix for the scikit-learn model
conf_matrix_sklearn = confusion_matrix(y, y_pred_sklearn)
print("Matriz de Confusión (scikit-learn):")
print(conf_matrix_sklearn)

# Calculate and print the classification report for the scikit-learn model
class_report_sklearn = classification_report(y, y_pred_sklearn)
print("\nReporte de Clasificación (scikit-learn):")
print(class_report_sklearn)

Matriz de Confusión (scikit-learn):
[[43  0  0]
 [ 0 41  0]
 [ 0  0 41]]

Reporte de Clasificación (scikit-learn):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        43
           1       1.00      1.00      1.00        41
           2       1.00      1.00      1.00        41

    accuracy                           1.00       125
   macro avg       1.00      1.00      1.00       125
weighted avg       1.00      1.00      1.00       125



In [53]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score

# Make predictions on the training data (you might want to use a separate test set)
y_pred = rf.predict(X)

# Calculate and print the confusion matrix
conf_matrix = confusion_matrix(y, y_pred)
print("Matriz de Confusión:")
print(conf_matrix)

# Calculate and print the classification report (includes precision, recall, f1-score)
class_report = classification_report(y, y_pred)
print("\nReporte de Clasificación:")
print(class_report)

Matriz de Confusión:
[[42  1  0]
 [ 0 39  2]
 [ 0  2 39]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99        43
           1       0.93      0.95      0.94        41
           2       0.95      0.95      0.95        41

    accuracy                           0.96       125
   macro avg       0.96      0.96      0.96       125
weighted avg       0.96      0.96      0.96       125



### Elemento 2 - Preguntas teóricas

**¿Qué diferencias cuantitativas y cualitativas se observan entre tu implementación y la de sklearn?**

Cuantitativamente, la implementación propia alcanzó un 96% de accuracy en el conjunto de entrenamiento y un 94.4% de accuracy promedio en validación cruzada (usando GridSearchCV), mientras que la versión de scikit-learn logró un 100% de accuracy en el mismo conjunto de entrenamiento, sugiriendo un posible sobreajuste o una mayor capacidad para capturar la complejidad de los datos de entrenamiento. Cualitativamente, la implementación propia se enfoca en replicar la lógica central del bagging y la votación usando `DecisionTreeClassifier` como base, mientras que scikit-learn es una implementación altamente optimizada (probablemente en Cython/C), más rápida, con más funcionalidades (como `n_jobs` para paralelización) y potencialmente con heurísticas internas más refinadas para la construcción de árboles y manejo de características.

**¿Cómo influyen los parámetros n_estimators y max_features en el desempeño del modelo?**

`n_estimators` controla el número de árboles en el bosque; incrementarlo generalmente reduce la varianza y mejora la estabilidad del modelo al promediar más predicciones, aunque el beneficio disminuye a partir de cierto punto y aumenta el costo computacional. `max_features` limita el número de características consideradas en cada división, incrementando la diversidad entre los árboles y reduciendo su correlación, ayudando a disminuir la varianza general del ensamble a costa de un posible ligero aumento en el sesgo de cada árbol individual; valores comunes como 'sqrt' buscan un equilibrio.

**¿Por qué el modelo de sklearn suele ser más rápido o más preciso?**

El modelo de scikit-learn suele ser más rápido porque está implementado con optimizaciones de bajo nivel (Cython/C) y permite paralelizar el entrenamiento de los árboles a través del parámetro `n_jobs`, aprovechando múltiples núcleos de CPU. Puede ser más preciso debido a optimizaciones en el algoritmo de construcción del árbol base, heurísticas más avanzadas para encontrar divisiones, manejo eficiente de datos o estrategias predeterminadas para parámetros no especificados; sin embargo, la implementación propia demostró un buen desempeño en validación cruzada (94.4%) y un buen accuracy general (96%).

**¿Tu implementación mantiene el mismo comportamiento al modificar la semilla aleatoria?**

No, la implementación no mantendrá exactamente el mismo comportamiento si se modifica la semilla aleatoria (`random_state`). Random Forest utiliza aleatoriedad en dos puntos clave: al crear las muestras bootstrap para cada árbol y al seleccionar el subconjunto de características (`max_features`) en cada nodo. Cambiar la semilla inicial alterará las muestras y las características seleccionadas, resultando en un bosque compuesto por árboles diferentes, lo que probablemente conducirá a ligeras variaciones en las métricas de desempeño y en las predicciones específicas, aunque las tendencias generales del modelo deberían ser similares.


## Elemento 3 - Creación y despliegue de la API

La creación de la API se encuentra en app.py

### Elemento 3 - Preguntas teóricas

**¿Qué ventajas ofrece exponer un modelo como servicio web?**

Exponer un modelo como servicio web permite que sus predicciones puedan ser utilizadas desde cualquier aplicación o sistema remoto mediante solicitudes HTTP, sin necesidad de acceder directamente al código o al entorno de desarrollo. Esto facilita la integración con otros servicios, la automatización de procesos y la actualización del modelo sin modificar las aplicaciones cliente. Además, ofrece escalabilidad, ya que el modelo puede atender múltiples peticiones simultáneas en la nube, y mantenibilidad al centralizar el modelo en un único punto de acceso.

**¿Qué riesgos o limitaciones pueden surgir si no se valida correctamente la entrada del usuario?**

Si no se valida adecuadamente la información que envía el usuario al endpoint `/predict`, pueden surgir errores de ejecución, resultados erróneos o vulnerabilidades de seguridad, como inyección de código o ataques de denegación de servicio (DoS). Una validación deficiente puede hacer que el servicio se vuelva inestable o que devuelva respuestas incoherentes, afectando la confiabilidad del modelo y la disponibilidad de la API.

**¿Por qué es importante incluir un endpoint de /health en una API?**

El endpoint `/health` es esencial para el monitoreo y diagnóstico del servicio, ya que permite confirmar rápidamente si la API está activa y funcionando correctamente. Este punto de control se usa comúnmente por plataformas de despliegue o sistemas de supervisión automática para detectar fallos, reiniciar el servicio si es necesario y garantizar que el modelo esté disponible. Además, simplifica la verificación manual por parte del equipo de desarrollo antes de realizar pruebas o actualizaciones.

**¿Cómo podrías garantizar que tu servicio mantenga disponibilidad bajo diferentes condiciones?**

Para mantener la disponibilidad, es necesario aplicar buenas prácticas de despliegue y diseño, validar correctamente las solicitudes, manejar excepciones para evitar caídas inesperadas, usar un servidor confiable en la nube y configurar reinicios automáticos en caso de error. También se puede mejorar la disponibilidad mediante pruebas de carga y concurrencia, optimización del rendimiento del modelo, y uso de mecanismos de escalado automático o balanceo de carga si se reciben muchas peticiones. Estas medidas aseguran que la API siga respondiendo aun bajo alta demanda o condiciones adversas.